In [1]:
import pandas as pd
import numpy as np

df = pd.read_csv('../datasets/Phishing_Email.csv')
df.head(2)

,Unnamed: 0,Email Text,Email Type
0,0,"re : 6 . 1100 , disc : uniformitarianism , re ...",Safe Email
1,1,the other side of * galicismos * * galicismo *...,Safe Email


In [2]:
df.dropna(inplace=True)

In [3]:
df.drop('Unnamed: 0', axis=1, inplace=True)
df.head()

,Email Text,Email Type
0,"re : 6 . 1100 , disc : uniformitarianism , re ...",Safe Email
1,the other side of * galicismos * * galicismo *...,Safe Email
2,re : equistar deal tickets are you still avail...,Safe Email
3,\nHello I am your hot lil horny toy.\n I am...,Phishing Email
4,software at incredibly low prices ( 86 % lower...,Phishing Email


In [4]:
df.rename(columns={"Email Text": "feature", "Email Type": "target"}, inplace=True)
df.head()

,feature,target
0,"re : 6 . 1100 , disc : uniformitarianism , re ...",Safe Email
1,the other side of * galicismos * * galicismo *...,Safe Email
2,re : equistar deal tickets are you still avail...,Safe Email
3,\nHello I am your hot lil horny toy.\n I am...,Phishing Email
4,software at incredibly low prices ( 86 % lower...,Phishing Email


In [5]:
"""
Now we do Data Preprocessing.
"""
import string, re, nltk
from string import punctuation
from nltk.tokenize import word_tokenize, RegexpTokenizer
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer

regexp = RegexpTokenizer("[\w']+")

def convert_to_lowercase(text):
    return text.lower()
def remove_whitespace(text):
    return text.strip()
def remove_punctuation(text):
    punct_str = string.punctuation
    punct_str = punct_str.replace("'", "") 
    return text.translate(str.maketrans("", "", punct_str))
def remove_html(text):
    html = re.compile(r'<.*?>')
    return html.sub(r'', text)
def remove_http(text):
    http = "https?://\S+|www\.\S+" 
    pattern = r"({})".format(http) 
    return re.sub(pattern, "", text)
# Stopwords
stops = stopwords.words("english") 
addstops = ["among", "onto", "shall", "thrice", "thus", "twice", "unto", "us", "would"] 
allstops = stops + addstops
def remove_stopwords(text):
    return " ".join([word for word in regexp.tokenize(text) if word not in allstops])
stemmer = PorterStemmer()
def text_stemmer(text):
    text_stem = " ".join([stemmer.stem(word) for word in regexp.tokenize(text)])
    return text_stem
def discard_non_alpha(text):
    word_list_non_alpha = [word for word in regexp.tokenize(text) if word.isalpha()]
    text_non_alpha = " ".join(word_list_non_alpha)
    return text_non_alpha


In [6]:
def text_normalizer(text):
    text = convert_to_lowercase(text)
    text = remove_whitespace(text)
    text = re.sub('\n' , '', text) 
    text = re.sub('\[.*?\]', '', text) 
    text = remove_http(text)
    text = remove_punctuation(text)
    text = remove_html(text)
    text = remove_stopwords(text)
    text = discard_non_alpha(text)
    return text

In [7]:
df['feature'] = df['feature'].apply(text_normalizer)
df['feature'][0]

'disc uniformitarianism sex lang dick hudson observations use aughter vocative thoughtprovoking sure fair attribute sons treated like senior relatives one thing normally use brother way aughter hard imagine natural class comprising senior relatives excluding brother another seem differences imagining distinction seems senior relative terms used wider variety contexts e g calling distance get someone attention hence beginning utterance whereas seems natural utterances like yes son hand son ones like son son help although perhaps latter ones completely impossible alexis mr'

In [8]:
from sklearn.model_selection import train_test_split
df_train, df_test = train_test_split(df, test_size=0.2, random_state=42)

In [9]:
import gzip
# Calculate compression ratio
def calculate_compression_ratio(text):
    compressed = len(gzip.compress(text.encode()))
    original = len(text.encode())
    compression_ratio = original / compressed
    return compression_ratio

# Print compression ratios for train and test sets
train_compression_ratio = calculate_compression_ratio(" ".join(df_train['feature']))
test_compression_ratio = calculate_compression_ratio(" ".join(df_test['feature']))

print("Compression Ratio - Train Set:", train_compression_ratio)
print("Compression Ratio - Test Set:", test_compression_ratio)


Compression Ratio - Train Set: 2.930071188289488
Compression Ratio - Test Set: 2.842831972126728


In [12]:
#Statistical data about the train and test sets
# N - Number of training and test set examples
N_train = len(df_train)
N_test = len(df_test)

# C - Number of classes
C = df['target'].nunique()

# Calculate average number of words (W) and characters (L) in each example
#text is tokenized by spaces
train_word_counts = df_train['feature'].apply(lambda x: len(x.split()))
test_word_counts = df_test['feature'].apply(lambda x: len(x.split()))
train_char_counts = df_train['feature'].apply(lambda x: len(x))
test_char_counts = df_test['feature'].apply(lambda x: len(x))
W_train = train_word_counts.mean()
W_test = test_word_counts.mean()
L_train = train_char_counts.mean()
L_test = test_char_counts.mean()

# V - Vocabulary size
#count unique words in the entire dataset
all_text = " ".join(df['feature'])
vocabulary = set(all_text.split())
V = len(vocabulary)

# Print the statistics
print("N - Number of training examples:", N_train)
print("N - Number of test examples:", N_test)
print("C - Number of classes:", C)
print("W - Average number of words in each example (Train):", W_train)
print("W - Average number of words in each example (Test):", W_test)
print("L - Average number of characters in each example (Train):", L_train)
print("L - Average number of characters in each example (Test):", L_test)
print("V - Vocabulary size:", V)

N - Number of training examples: 14907
N - Number of test examples: 3727
C - Number of classes: 2
W - Average number of words in each example (Train): 247.98577849332528
W - Average number of words in each example (Test): 155.9922189428495
L - Average number of characters in each example (Train): 1849.8378614073924
L - Average number of characters in each example (Test): 1149.4926214113227
V - Vocabulary size: 174180


In [10]:
import gzip

#calculate normalized compression distance (NCD)
def calculate_ncd(x1, x2):
    Cx1 = len(gzip.compress(x1.encode()))
    Cx2 = len(gzip.compress(x2.encode()))
    x1x2 = " ".join([x1, x2])
    Cx1x2 = len(gzip.compress(x1x2.encode()))
    
    ncd = (Cx1x2 - min(Cx1, Cx2)) / max(Cx1, Cx2)
    return ncd

In [11]:
#normale Gzip-Klassifikationsvariante

from tqdm import tqdm
from collections import Counter

k = 2

predicted_classes = []

for row_test in tqdm(df_test.iterrows(), total=df_test.shape[0]):
    test_text = row_test[1]["feature"]
    test_label = row_test[1]["target"]
    c_test_text = len(gzip.compress(test_text.encode()))
    distance_from_test_instance = []
    
    for row_train in df_train.iterrows():
        train_text = row_train[1]["feature"]
        train_label = row_train[1]["target"]
        c_train_text = len(gzip.compress(train_text.encode()))
        
        train_plus_test = " ".join([test_text, train_text])
        c_train_plus_test = len(gzip.compress(train_plus_test.encode()))
        
        ncd = ( (c_train_plus_test - min(c_train_text, c_test_text))
                / max(c_test_text, c_train_text) )
        distance_from_test_instance.append(ncd)
        
    sorted_idx = np.argsort(np.array(distance_from_test_instance))
    
    top_k_class = list(df_train.iloc[sorted_idx[:k]]["target"].values)
    predicted_class = max(set(top_k_class), key=top_k_class.count)
    #top_k_class = df_train.iloc[sorted_idx[:k]]["target"].values
    #predicted_class = np.argmax(np.bincount(top_k_class))
    
    predicted_classes.append(predicted_class)
     
print("Accuracy:", np.mean(np.array(predicted_classes) == df_test["target"].values))

  0%|          | 1/3727 [00:05<6:02:45,  5.84s/it]


KeyboardInterrupt: 

In [ ]:
"""#with Tie-breaking fix (MORE ACCURACY)

from tqdm import tqdm
from collections import Counter

k = 2

predicted_classes = []

for row_test in tqdm(df_test.iterrows(), total=df_test.shape[0]):
    test_text = row_test[1]["feature"]
    test_label = row_test[1]["target"]
    c_test_text = len(gzip.compress(test_text.encode()))
    distance_from_test_instance = []
    
    for row_train in df_train.iterrows():
        train_text = row_train[1]["feature"]
        train_label = row_train[1]["target"]
        c_train_text = len(gzip.compress(train_text.encode()))
        
        train_plus_test = " ".join([test_text, train_text])
        c_train_plus_test = len(gzip.compress(train_plus_test.encode()))
        
        ncd = ( (c_train_plus_test - min(c_train_text, c_test_text))
                / max(c_test_text, c_train_text) )
        distance_from_test_instance.append(ncd)
        
    sorted_idx = np.argsort(np.array(distance_from_test_instance))
    top_k_class = np.array(df_train["target"])[sorted_idx[:k]]
    predicted_class = Counter(top_k_class).most_common()[0][0]
    
    predicted_classes.append(predicted_class)
        
print("Accuracy:", np.mean(np.array(predicted_classes) == df_test["target"].values))"""

In [ ]:
from sklearn.metrics import classification_report, accuracy_score

#true labels for the test set
true_labels = df_test['target']

# Compute the classification report
classification_report_output = classification_report(true_labels, predicted_classes)

# Print the classification report
print(classification_report_output)